In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["dir", "/b", "/a-d","input"], shell=True).decode("utf8"))

# Any results you write to the current directory are saved as output.

BBTeamSeasonStats.csv
RegularSeasonCompactResults.csv
RegularSeasonDetailedResults.csv
SampleSubmission.csv
Seasons.csv
Teams.csv
TourneyCompactResults.csv
TourneyDetailedResults.csv
TourneySeeds.csv
TourneySlots.csv



In [2]:
data_dir = 'input/'
df_seeds = pd.read_csv(data_dir + 'TourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'TourneyCompactResults.csv')
df_BBSeason = pd.read_csv(data_dir + 'BBTeamSeasonStats.csv')

In [3]:
df_BBSeason.head()

,Season,Team,WinLossPct,AvgPts,PtDiff,AvgRebounds,AvgAssists,AvgTurnovers,AvgFouls,FGPct,FTPct,ThPct,AvgStealsPlusBlocks,AvgPtsAgainst,OppFGPct,OppThPct
0,2003,1291,0.41,63,-166,35,11,15,21,0.39,0.70,0.32,11,69,0.45,0.40
1,2007,1144,0.41,76,-95,38,15,17,19,0.44,0.72,0.33,12,79,0.47,0.34
2,2008,1208,0.50,67,29,39,11,14,18,0.43,0.67,0.33,10,66,0.43,0.31
3,2013,1386,0.58,67,107,35,14,11,13,0.44,0.70,0.34,10,63,0.41,0.31
4,2014,1450,0.30,61,-177,35,10,11,18,0.40,0.66,0.33,8,67,0.46,0.37


In [4]:
df_tour.drop(labels=['Daynum', 'Wscore', 'Lscore', 'Wloc', 'Numot'], inplace=True, axis=1)

In [5]:
def seed_to_int(seed):
    """Get just the digits from the seeding. Return as int"""
    s_int = int(seed[1:3])
    return s_int
df_seeds['n_seed'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label

In [6]:
df_winseeds = df_seeds.rename(columns={'Team':'Wteam', 'n_seed':'win_seed'})
df_lossseeds = df_seeds.rename(columns={'Team':'Lteam', 'n_seed':'loss_seed'})

In [7]:
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'Wteam'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'Lteam'])
df_concat['seed_diff'] = df_concat.win_seed - df_concat.loss_seed

In [8]:
df_concat.head()

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff
0,1985,1116,1234,9,8,1
1,1985,1120,1345,11,6,5
2,1985,1207,1250,1,16,-15
3,1985,1229,1425,9,8,1
4,1985,1242,1325,3,14,-11


In [9]:
##Make a df to play with so I don't mess up df_concat
##We need to get other stats and differentials into this dataframe the same way as Seed_diff is there.
df_concat2 = df_concat

In [10]:
df_concat2.head()

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff
0,1985,1116,1234,9,8,1
1,1985,1120,1345,11,6,5
2,1985,1207,1250,1,16,-15
3,1985,1229,1425,9,8,1
4,1985,1242,1325,3,14,-11


In [11]:
df_BBSeason.head()

,Season,Team,WinLossPct,AvgPts,PtDiff,AvgRebounds,AvgAssists,AvgTurnovers,AvgFouls,FGPct,FTPct,ThPct,AvgStealsPlusBlocks,AvgPtsAgainst,OppFGPct,OppThPct
0,2003,1291,0.41,63,-166,35,11,15,21,0.39,0.70,0.32,11,69,0.45,0.40
1,2007,1144,0.41,76,-95,38,15,17,19,0.44,0.72,0.33,12,79,0.47,0.34
2,2008,1208,0.50,67,29,39,11,14,18,0.43,0.67,0.33,10,66,0.43,0.31
3,2013,1386,0.58,67,107,35,14,11,13,0.44,0.70,0.34,10,63,0.41,0.31
4,2014,1450,0.30,61,-177,35,10,11,18,0.40,0.66,0.33,8,67,0.46,0.37


In [12]:
df_winseeds = df_seeds.rename(columns={'Team':'Wteam', 'n_seed':'win_seed'})
df_lossseeds = df_seeds.rename(columns={'Team':'Lteam', 'n_seed':'loss_seed'})

In [13]:
df_winseeds.head()

,Season,Wteam,win_seed
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [14]:
df_WinStats = df_BBSeason.rename(columns={'Team':'Wteam'})

In [15]:
df_WinStats.head()

,Season,Wteam,WinLossPct,AvgPts,PtDiff,AvgRebounds,AvgAssists,AvgTurnovers,AvgFouls,FGPct,FTPct,ThPct,AvgStealsPlusBlocks,AvgPtsAgainst,OppFGPct,OppThPct
0,2003,1291,0.41,63,-166,35,11,15,21,0.39,0.70,0.32,11,69,0.45,0.40
1,2007,1144,0.41,76,-95,38,15,17,19,0.44,0.72,0.33,12,79,0.47,0.34
2,2008,1208,0.50,67,29,39,11,14,18,0.43,0.67,0.33,10,66,0.43,0.31
3,2013,1386,0.58,67,107,35,14,11,13,0.44,0.70,0.34,10,63,0.41,0.31
4,2014,1450,0.30,61,-177,35,10,11,18,0.40,0.66,0.33,8,67,0.46,0.37


In [16]:
df_WinStats2 = pd.merge(left=df_winseeds, right=df_WinStats, on=['Season', 'Wteam'])

In [17]:
df_LossStats = df_BBSeason.rename(columns={'Team':'Lteam'})

In [18]:
df_LossStats2 = pd.merge(left=df_lossseeds, right=df_LossStats, on=['Season', 'Lteam'])

In [19]:
df_LossStats.head()

,Season,Lteam,WinLossPct,AvgPts,PtDiff,AvgRebounds,AvgAssists,AvgTurnovers,AvgFouls,FGPct,FTPct,ThPct,AvgStealsPlusBlocks,AvgPtsAgainst,OppFGPct,OppThPct
0,2003,1291,0.41,63,-166,35,11,15,21,0.39,0.70,0.32,11,69,0.45,0.40
1,2007,1144,0.41,76,-95,38,15,17,19,0.44,0.72,0.33,12,79,0.47,0.34
2,2008,1208,0.50,67,29,39,11,14,18,0.43,0.67,0.33,10,66,0.43,0.31
3,2013,1386,0.58,67,107,35,14,11,13,0.44,0.70,0.34,10,63,0.41,0.31
4,2014,1450,0.30,61,-177,35,10,11,18,0.40,0.66,0.33,8,67,0.46,0.37


In [20]:
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'Wteam'])

In [21]:
# Use .add_prefix to add a prefix to each of the columns in your data frame
# Add W_ to each of the columns in the WinStats2 data frame
df_WinStatsPrefixed = df_WinStats2.add_prefix("W_")

# Add L_ to each of the columns in the LossStats2 data frame
df_LossStatsPrefixed = df_LossStats2.add_prefix("L_")

df_WinStatsPrefixed.head()

,W_Season,W_Wteam,W_win_seed,W_WinLossPct,W_AvgPts,W_PtDiff,W_AvgRebounds,W_AvgAssists,W_AvgTurnovers,W_AvgFouls,W_FGPct,W_FTPct,W_ThPct,W_AvgStealsPlusBlocks,W_AvgPtsAgainst,W_OppFGPct,W_OppThPct
0,2003,1328,1,0.80,71,330,37,14,11,18,0.45,0.71,0.39,10,60,0.40,0.33
1,2003,1448,2,0.83,78,313,42,14,15,18,0.46,0.75,0.34,10,67,0.40,0.34
2,2003,1393,3,0.83,80,296,41,14,13,16,0.47,0.69,0.33,15,69,0.39,0.31
3,2003,1257,4,0.80,81,401,38,16,13,22,0.46,0.70,0.36,11,68,0.40,0.30
4,2003,1280,5,0.70,70,300,37,14,15,15,0.47,0.68,0.34,12,60,0.40,0.33


In [22]:
df_WinStatsPrefixed.head()

,W_Season,W_Wteam,W_win_seed,W_WinLossPct,W_AvgPts,W_PtDiff,W_AvgRebounds,W_AvgAssists,W_AvgTurnovers,W_AvgFouls,W_FGPct,W_FTPct,W_ThPct,W_AvgStealsPlusBlocks,W_AvgPtsAgainst,W_OppFGPct,W_OppThPct
0,2003,1328,1,0.80,71,330,37,14,11,18,0.45,0.71,0.39,10,60,0.40,0.33
1,2003,1448,2,0.83,78,313,42,14,15,18,0.46,0.75,0.34,10,67,0.40,0.34
2,2003,1393,3,0.83,80,296,41,14,13,16,0.47,0.69,0.33,15,69,0.39,0.31
3,2003,1257,4,0.80,81,401,38,16,13,22,0.46,0.70,0.36,11,68,0.40,0.30
4,2003,1280,5,0.70,70,300,37,14,15,15,0.47,0.68,0.34,12,60,0.40,0.33


In [23]:
df_WinStatsPrefixed = df_WinStatsPrefixed.rename(columns={'W_Season':'Season', 'W_Wteam':'Wteam'})
df_LossStatsPrefixed = df_LossStatsPrefixed.rename(columns={'L_Season':'Season', 'L_Lteam':'Lteam'})

In [24]:
df_LossStatsPrefixed.head()

,Season,Lteam,L_loss_seed,L_WinLossPct,L_AvgPts,L_PtDiff,L_AvgRebounds,L_AvgAssists,L_AvgTurnovers,L_AvgFouls,L_FGPct,L_FTPct,L_ThPct,L_AvgStealsPlusBlocks,L_AvgPtsAgainst,L_OppFGPct,L_OppThPct
0,2003,1328,1,0.80,71,330,37,14,11,18,0.45,0.71,0.39,10,60,0.40,0.33
1,2003,1448,2,0.83,78,313,42,14,15,18,0.46,0.75,0.34,10,67,0.40,0.34
2,2003,1393,3,0.83,80,296,41,14,13,16,0.47,0.69,0.33,15,69,0.39,0.31
3,2003,1257,4,0.80,81,401,38,16,13,22,0.46,0.70,0.36,11,68,0.40,0.30
4,2003,1280,5,0.70,70,300,37,14,15,15,0.47,0.68,0.34,12,60,0.40,0.33


In [25]:
df_concat2 = pd.merge(left=df_concat2, right=df_LossStatsPrefixed,  on=['Season', 'Lteam'])
df_concat3 = pd.merge(left=df_concat2, right=df_WinStatsPrefixed,  on=['Season', 'Wteam'])

In [26]:
##Use this dataframe to calculate stats for wins and losses in separate dataframes by just reversing the calculations.
##For example, for the winning team, calculate avg_point_diff as (W_AvgPts_x - L_AvgPts_x), 
##but in another dataframe for the losers calculate it as (L_AvgPts_x - W_AvgPts_x).
##Once all calculations are made in separate dataframes add the 'Result' column to each (like in this original notebook)
##and finally UNION all records.  This way, we have examples of both wins and losses with appropriate stats.
##Once this is all complete, drop all columns exept for Season, Wteam, LTeam, seed_diff, Result, and all calculated diffs.
##THAT should be our final data set.

df_concat3.head()

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff,L_loss_seed,L_WinLossPct,L_AvgPts,L_PtDiff,...,W_AvgAssists,W_AvgTurnovers,W_AvgFouls,W_FGPct,W_FTPct,W_ThPct,W_AvgStealsPlusBlocks,W_AvgPtsAgainst,W_OppFGPct,W_OppThPct
0,2003,1421,1411,16,16,0,16,0.60,72,59,...,13,16,19,0.43,0.76,0.36,10,78,0.46,0.37
1,2003,1112,1436,1,16,-15,16,0.66,67,135,...,17,14,17,0.46,0.70,0.35,12,70,0.41,0.32
2,2003,1112,1211,1,9,-8,9,0.74,77,275,...,17,14,17,0.46,0.70,0.35,12,70,0.41,0.32
3,2003,1112,1323,1,5,-4,5,0.71,80,276,...,17,14,17,0.46,0.70,0.35,12,70,0.41,0.32
4,2003,1113,1272,10,7,3,7,0.79,74,252,...,15,14,19,0.48,0.67,0.32,9,69,0.45,0.34


In [29]:
# Winner Stats
# if you set a dataframe = to another dataframe it will modify the the original data frame.  
# Use copy to preserve and not accidentally do bad things and upset yourself later
df_WinnerStats = df_concat3.copy(deep=True)

# Calculate Differentials
df_WinnerStats['diff_WinLossPct'] = df_WinnerStats.W_WinLossPct - df_WinnerStats.L_WinLossPct
df_WinnerStats['diff_AvgPts'] = df_WinnerStats.W_AvgPts - df_WinnerStats.L_AvgPts
df_WinnerStats['diff_PtDiff'] = df_WinnerStats.W_PtDiff - df_WinnerStats.L_PtDiff
df_WinnerStats['diff_AvgRebounds'] = df_WinnerStats.W_AvgRebounds - df_WinnerStats.L_AvgRebounds
df_WinnerStats['diff_AvgAssists'] = df_WinnerStats.W_AvgAssists - df_WinnerStats.L_AvgAssists
df_WinnerStats['diff_AvgTurnovers'] = df_WinnerStats.W_AvgTurnovers - df_WinnerStats.L_AvgTurnovers
df_WinnerStats['diff_AvgFouls'] = df_WinnerStats.W_AvgFouls - df_WinnerStats.L_AvgFouls
df_WinnerStats['diff_FGPct'] = df_WinnerStats.W_FGPct - df_WinnerStats.L_FGPct
df_WinnerStats['diff_FTPct'] = df_WinnerStats.W_FTPct - df_WinnerStats.L_FTPct
df_WinnerStats['diff_ThPct'] = df_WinnerStats.W_ThPct - df_WinnerStats.L_ThPct
df_WinnerStats['diff_AvgStealsPlusBlocks'] = df_WinnerStats.W_AvgStealsPlusBlocks - df_WinnerStats.L_AvgStealsPlusBlocks
df_WinnerStats['diff_AvgPtsAgainst'] = df_WinnerStats.W_AvgPtsAgainst - df_WinnerStats.L_AvgPtsAgainst
df_WinnerStats['diff_OppFGPct'] = df_WinnerStats.W_OppFGPct - df_WinnerStats.L_OppFGPct
df_WinnerStats['diff_OppThPct'] = df_WinnerStats.W_OppThPct - df_WinnerStats.L_OppThPct
df_WinnerStats['Result'] = 1




In [30]:
df_WinnerStats

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff,L_loss_seed,L_WinLossPct,L_AvgPts,L_PtDiff,...,diff_AvgTurnovers,diff_AvgFouls,diff_FGPct,diff_FTPct,diff_ThPct,diff_AvgStealsPlusBlocks,diff_AvgPtsAgainst,diff_OppFGPct,diff_OppThPct,Result
0,2003,1421,1411,16,16,0,16,0.60,72,59,...,1,1,-0.02,0.14,0.04,2,8,0.04,0.04,1
1,2003,1112,1436,1,16,-15,16,0.66,67,135,...,0,2,0.02,0.04,0.01,3,7,0.00,-0.01,1
2,2003,1112,1211,1,9,-8,9,0.74,77,275,...,0,-1,-0.01,-0.02,-0.03,2,2,-0.01,-0.05,1
3,2003,1112,1323,1,5,-4,5,0.71,80,276,...,2,1,0.01,-0.06,-0.03,-1,-1,0.00,-0.05,1
4,2003,1113,1272,10,7,3,7,0.79,74,252,...,1,1,0.04,0.02,-0.03,-3,4,0.05,0.02,1
5,2003,1141,1166,11,6,5,6,0.88,79,492,...,5,3,0.01,0.08,-0.01,-1,9,0.02,0.02,1
6,2003,1143,1301,8,9,-1,9,0.60,72,132,...,0,-1,0.01,-0.08,0.03,-1,1,-0.01,-0.01,1
7,2003,1163,1140,5,12,-7,12,0.74,72,272,...,2,-3,0.00,-0.08,0.01,4,8,-0.03,0.01,1
8,2003,1163,1390,5,4,1,4,0.74,72,164,...,1,0,0.03,-0.01,0.03,5,5,-0.03,0.02,1
9,2003,1181,1161,3,14,-11,14,0.57,74,40,...,-2,1,-0.06,0.01,0.00,4,-2,-0.01,0.00,1


In [ ]:
df_WinnerStats.columns

In [32]:
# Loser Stats
df_LoserStats = df_concat3.copy(deep=True)

# Calculate Differentials
df_LoserStats['diff_WinLossPct'] = df_WinnerStats.L_WinLossPct - df_WinnerStats.W_WinLossPct
df_LoserStats['diff_AvgPts'] = df_WinnerStats.L_AvgPts - df_WinnerStats.W_AvgPts
df_LoserStats['diff_PtDiff'] = df_WinnerStats.L_PtDiff - df_WinnerStats.W_PtDiff
df_LoserStats['diff_AvgRebounds'] = df_WinnerStats.L_AvgRebounds - df_WinnerStats.W_AvgRebounds
df_LoserStats['diff_AvgAssists'] = df_WinnerStats.L_AvgAssists - df_WinnerStats.W_AvgAssists
df_LoserStats['diff_AvgTurnovers'] = df_WinnerStats.L_AvgTurnovers - df_WinnerStats.W_AvgTurnovers
df_LoserStats['diff_AvgFouls'] = df_WinnerStats.L_AvgFouls - df_WinnerStats.W_AvgFouls
df_LoserStats['diff_FGPct'] = df_WinnerStats.L_FGPct - df_WinnerStats.W_FGPct
df_LoserStats['diff_FTPct'] = df_WinnerStats.L_FTPct - df_WinnerStats.W_FTPct
df_LoserStats['diff_ThPct'] = df_WinnerStats.L_ThPct - df_WinnerStats.W_ThPct
df_LoserStats['diff_AvgStealsPlusBlocks'] = df_WinnerStats.L_AvgStealsPlusBlocks - df_WinnerStats.W_AvgStealsPlusBlocks
df_LoserStats['diff_AvgPtsAgainst'] = df_WinnerStats.L_AvgPtsAgainst - df_WinnerStats.W_AvgPtsAgainst
df_LoserStats['diff_OppFGPct'] = df_WinnerStats.L_OppFGPct - df_WinnerStats.W_OppFGPct
df_LoserStats['diff_OppThPct'] = df_WinnerStats.L_OppThPct - df_WinnerStats.W_OppThPct
df_LoserStats['Result'] = 0

In [35]:
df_LoserStats

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff,L_loss_seed,L_WinLossPct,L_AvgPts,L_PtDiff,...,diff_AvgTurnovers,diff_AvgFouls,diff_FGPct,diff_FTPct,diff_ThPct,diff_AvgStealsPlusBlocks,diff_AvgPtsAgainst,diff_OppFGPct,diff_OppThPct,Result
0,2003,1421,1411,16,16,0,16,0.60,72,59,...,-1,-1,0.02,-0.14,-0.04,-2,-8,-0.04,-0.04,0
1,2003,1112,1436,1,16,-15,16,0.66,67,135,...,0,-2,-0.02,-0.04,-0.01,-3,-7,0.00,0.01,0
2,2003,1112,1211,1,9,-8,9,0.74,77,275,...,0,1,0.01,0.02,0.03,-2,-2,0.01,0.05,0
3,2003,1112,1323,1,5,-4,5,0.71,80,276,...,-2,-1,-0.01,0.06,0.03,1,1,0.00,0.05,0
4,2003,1113,1272,10,7,3,7,0.79,74,252,...,-1,-1,-0.04,-0.02,0.03,3,-4,-0.05,-0.02,0
5,2003,1141,1166,11,6,5,6,0.88,79,492,...,-5,-3,-0.01,-0.08,0.01,1,-9,-0.02,-0.02,0
6,2003,1143,1301,8,9,-1,9,0.60,72,132,...,0,1,-0.01,0.08,-0.03,1,-1,0.01,0.01,0
7,2003,1163,1140,5,12,-7,12,0.74,72,272,...,-2,3,0.00,0.08,-0.01,-4,-8,0.03,-0.01,0
8,2003,1163,1390,5,4,1,4,0.74,72,164,...,-1,0,-0.03,0.01,-0.03,-5,-5,0.03,-0.02,0
9,2003,1181,1161,3,14,-11,14,0.57,74,40,...,2,-1,0.06,-0.01,0.00,-4,2,0.01,0.00,0


In [36]:
df_LoserStats.columns

Index(['Season', 'Wteam', 'Lteam', 'win_seed', 'loss_seed', 'seed_diff',
       'L_loss_seed', 'L_WinLossPct', 'L_AvgPts', 'L_PtDiff', 'L_AvgRebounds',
       'L_AvgAssists', 'L_AvgTurnovers', 'L_AvgFouls', 'L_FGPct', 'L_FTPct',
       'L_ThPct', 'L_AvgStealsPlusBlocks', 'L_AvgPtsAgainst', 'L_OppFGPct',
       'L_OppThPct', 'W_win_seed', 'W_WinLossPct', 'W_AvgPts', 'W_PtDiff',
       'W_AvgRebounds', 'W_AvgAssists', 'W_AvgTurnovers', 'W_AvgFouls',
       'W_FGPct', 'W_FTPct', 'W_ThPct', 'W_AvgStealsPlusBlocks',
       'W_AvgPtsAgainst', 'W_OppFGPct', 'W_OppThPct', 'diff_WinLossPct',
       'diff_AvgPts', 'diff_PtDiff', 'diff_AvgRebounds', 'diff_AvgAssists',
       'diff_AvgTurnovers', 'diff_AvgFouls', 'diff_FGPct', 'diff_FTPct',
       'diff_ThPct', 'diff_AvgStealsPlusBlocks', 'diff_AvgPtsAgainst',
       'diff_OppFGPct', 'diff_OppThPct', 'Result'],
      dtype='object')

In [37]:
df_WinnerStats

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff,L_loss_seed,L_WinLossPct,L_AvgPts,L_PtDiff,...,diff_AvgTurnovers,diff_AvgFouls,diff_FGPct,diff_FTPct,diff_ThPct,diff_AvgStealsPlusBlocks,diff_AvgPtsAgainst,diff_OppFGPct,diff_OppThPct,Result
0,2003,1421,1411,16,16,0,16,0.60,72,59,...,1,1,-0.02,0.14,0.04,2,8,0.04,0.04,1
1,2003,1112,1436,1,16,-15,16,0.66,67,135,...,0,2,0.02,0.04,0.01,3,7,0.00,-0.01,1
2,2003,1112,1211,1,9,-8,9,0.74,77,275,...,0,-1,-0.01,-0.02,-0.03,2,2,-0.01,-0.05,1
3,2003,1112,1323,1,5,-4,5,0.71,80,276,...,2,1,0.01,-0.06,-0.03,-1,-1,0.00,-0.05,1
4,2003,1113,1272,10,7,3,7,0.79,74,252,...,1,1,0.04,0.02,-0.03,-3,4,0.05,0.02,1
5,2003,1141,1166,11,6,5,6,0.88,79,492,...,5,3,0.01,0.08,-0.01,-1,9,0.02,0.02,1
6,2003,1143,1301,8,9,-1,9,0.60,72,132,...,0,-1,0.01,-0.08,0.03,-1,1,-0.01,-0.01,1
7,2003,1163,1140,5,12,-7,12,0.74,72,272,...,2,-3,0.00,-0.08,0.01,4,8,-0.03,0.01,1
8,2003,1163,1390,5,4,1,4,0.74,72,164,...,1,0,0.03,-0.01,0.03,5,5,-0.03,0.02,1
9,2003,1181,1161,3,14,-11,14,0.57,74,40,...,-2,1,-0.06,0.01,0.00,4,-2,-0.01,0.00,1


In [38]:
df_LoserStats

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff,L_loss_seed,L_WinLossPct,L_AvgPts,L_PtDiff,...,diff_AvgTurnovers,diff_AvgFouls,diff_FGPct,diff_FTPct,diff_ThPct,diff_AvgStealsPlusBlocks,diff_AvgPtsAgainst,diff_OppFGPct,diff_OppThPct,Result
0,2003,1421,1411,16,16,0,16,0.60,72,59,...,-1,-1,0.02,-0.14,-0.04,-2,-8,-0.04,-0.04,0
1,2003,1112,1436,1,16,-15,16,0.66,67,135,...,0,-2,-0.02,-0.04,-0.01,-3,-7,0.00,0.01,0
2,2003,1112,1211,1,9,-8,9,0.74,77,275,...,0,1,0.01,0.02,0.03,-2,-2,0.01,0.05,0
3,2003,1112,1323,1,5,-4,5,0.71,80,276,...,-2,-1,-0.01,0.06,0.03,1,1,0.00,0.05,0
4,2003,1113,1272,10,7,3,7,0.79,74,252,...,-1,-1,-0.04,-0.02,0.03,3,-4,-0.05,-0.02,0
5,2003,1141,1166,11,6,5,6,0.88,79,492,...,-5,-3,-0.01,-0.08,0.01,1,-9,-0.02,-0.02,0
6,2003,1143,1301,8,9,-1,9,0.60,72,132,...,0,1,-0.01,0.08,-0.03,1,-1,0.01,0.01,0
7,2003,1163,1140,5,12,-7,12,0.74,72,272,...,-2,3,0.00,0.08,-0.01,-4,-8,0.03,-0.01,0
8,2003,1163,1390,5,4,1,4,0.74,72,164,...,-1,0,-0.03,0.01,-0.03,-5,-5,0.03,-0.02,0
9,2003,1181,1161,3,14,-11,14,0.57,74,40,...,2,-1,0.06,-0.01,0.00,-4,2,0.01,0.00,0


In [39]:
df_LoserStats

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff,L_loss_seed,L_WinLossPct,L_AvgPts,L_PtDiff,...,diff_AvgTurnovers,diff_AvgFouls,diff_FGPct,diff_FTPct,diff_ThPct,diff_AvgStealsPlusBlocks,diff_AvgPtsAgainst,diff_OppFGPct,diff_OppThPct,Result
0,2003,1421,1411,16,16,0,16,0.60,72,59,...,-1,-1,0.02,-0.14,-0.04,-2,-8,-0.04,-0.04,0
1,2003,1112,1436,1,16,-15,16,0.66,67,135,...,0,-2,-0.02,-0.04,-0.01,-3,-7,0.00,0.01,0
2,2003,1112,1211,1,9,-8,9,0.74,77,275,...,0,1,0.01,0.02,0.03,-2,-2,0.01,0.05,0
3,2003,1112,1323,1,5,-4,5,0.71,80,276,...,-2,-1,-0.01,0.06,0.03,1,1,0.00,0.05,0
4,2003,1113,1272,10,7,3,7,0.79,74,252,...,-1,-1,-0.04,-0.02,0.03,3,-4,-0.05,-0.02,0
5,2003,1141,1166,11,6,5,6,0.88,79,492,...,-5,-3,-0.01,-0.08,0.01,1,-9,-0.02,-0.02,0
6,2003,1143,1301,8,9,-1,9,0.60,72,132,...,0,1,-0.01,0.08,-0.03,1,-1,0.01,0.01,0
7,2003,1163,1140,5,12,-7,12,0.74,72,272,...,-2,3,0.00,0.08,-0.01,-4,-8,0.03,-0.01,0
8,2003,1163,1390,5,4,1,4,0.74,72,164,...,-1,0,-0.03,0.01,-0.03,-5,-5,0.03,-0.02,0
9,2003,1181,1161,3,14,-11,14,0.57,74,40,...,2,-1,0.06,-0.01,0.00,-4,2,0.01,0.00,0


In [40]:
concats = [df_WinnerStats, df_LoserStats]
df_Concats = df_WinnerStats.append(df_LoserStats)
df_Concats

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff,L_loss_seed,L_WinLossPct,L_AvgPts,L_PtDiff,...,diff_AvgTurnovers,diff_AvgFouls,diff_FGPct,diff_FTPct,diff_ThPct,diff_AvgStealsPlusBlocks,diff_AvgPtsAgainst,diff_OppFGPct,diff_OppThPct,Result
0,2003,1421,1411,16,16,0,16,0.60,72,59,...,1,1,-0.02,0.14,0.04,2,8,0.04,0.04,1
1,2003,1112,1436,1,16,-15,16,0.66,67,135,...,0,2,0.02,0.04,0.01,3,7,0.00,-0.01,1
2,2003,1112,1211,1,9,-8,9,0.74,77,275,...,0,-1,-0.01,-0.02,-0.03,2,2,-0.01,-0.05,1
3,2003,1112,1323,1,5,-4,5,0.71,80,276,...,2,1,0.01,-0.06,-0.03,-1,-1,0.00,-0.05,1
4,2003,1113,1272,10,7,3,7,0.79,74,252,...,1,1,0.04,0.02,-0.03,-3,4,0.05,0.02,1
5,2003,1141,1166,11,6,5,6,0.88,79,492,...,5,3,0.01,0.08,-0.01,-1,9,0.02,0.02,1
6,2003,1143,1301,8,9,-1,9,0.60,72,132,...,0,-1,0.01,-0.08,0.03,-1,1,-0.01,-0.01,1
7,2003,1163,1140,5,12,-7,12,0.74,72,272,...,2,-3,0.00,-0.08,0.01,4,8,-0.03,0.01,1
8,2003,1163,1390,5,4,1,4,0.74,72,164,...,1,0,0.03,-0.01,0.03,5,5,-0.03,0.02,1
9,2003,1181,1161,3,14,-11,14,0.57,74,40,...,-2,1,-0.06,0.01,0.00,4,-2,-0.01,0.00,1


In [41]:
df_Concats.columns

Index(['Season', 'Wteam', 'Lteam', 'win_seed', 'loss_seed', 'seed_diff',
       'L_loss_seed', 'L_WinLossPct', 'L_AvgPts', 'L_PtDiff', 'L_AvgRebounds',
       'L_AvgAssists', 'L_AvgTurnovers', 'L_AvgFouls', 'L_FGPct', 'L_FTPct',
       'L_ThPct', 'L_AvgStealsPlusBlocks', 'L_AvgPtsAgainst', 'L_OppFGPct',
       'L_OppThPct', 'W_win_seed', 'W_WinLossPct', 'W_AvgPts', 'W_PtDiff',
       'W_AvgRebounds', 'W_AvgAssists', 'W_AvgTurnovers', 'W_AvgFouls',
       'W_FGPct', 'W_FTPct', 'W_ThPct', 'W_AvgStealsPlusBlocks',
       'W_AvgPtsAgainst', 'W_OppFGPct', 'W_OppThPct', 'diff_WinLossPct',
       'diff_AvgPts', 'diff_PtDiff', 'diff_AvgRebounds', 'diff_AvgAssists',
       'diff_AvgTurnovers', 'diff_AvgFouls', 'diff_FGPct', 'diff_FTPct',
       'diff_ThPct', 'diff_AvgStealsPlusBlocks', 'diff_AvgPtsAgainst',
       'diff_OppFGPct', 'diff_OppThPct', 'Result'],
      dtype='object')

In [42]:
df_Concats.drop(labels=['L_loss_seed', 'L_WinLossPct', 'L_AvgPts', 'L_PtDiff', 'L_AvgRebounds',
       'L_AvgAssists', 'L_AvgTurnovers', 'L_AvgFouls', 'L_FGPct', 'L_FTPct',
       'L_ThPct', 'L_AvgStealsPlusBlocks', 'L_AvgPtsAgainst', 'L_OppFGPct',
       'L_OppThPct', 'W_win_seed', 'W_WinLossPct', 'W_AvgPts', 'W_PtDiff',
       'W_AvgRebounds', 'W_AvgAssists', 'W_AvgTurnovers', 'W_AvgFouls',
       'W_FGPct', 'W_FTPct', 'W_ThPct', 'W_AvgStealsPlusBlocks',
       'W_AvgPtsAgainst', 'W_OppFGPct', 'W_OppThPct'],inplace=True, axis=1)



In [43]:
df_Concats

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff,diff_WinLossPct,diff_AvgPts,diff_PtDiff,diff_AvgRebounds,...,diff_AvgTurnovers,diff_AvgFouls,diff_FGPct,diff_FTPct,diff_ThPct,diff_AvgStealsPlusBlocks,diff_AvgPtsAgainst,diff_OppFGPct,diff_OppThPct,Result
0,2003,1421,1411,16,16,0,-0.15,-1,-269,-2,...,1,1,-0.02,0.14,0.04,2,8,0.04,0.04,1
1,2003,1112,1436,1,16,-15,0.23,18,284,4,...,0,2,0.02,0.04,0.01,3,7,0.00,-0.01,1
2,2003,1112,1211,1,9,-8,0.15,8,144,6,...,0,-1,-0.01,-0.02,-0.03,2,2,-0.01,-0.05,1
3,2003,1112,1323,1,5,-4,0.18,5,143,4,...,2,1,0.01,-0.06,-0.03,-1,-1,0.00,-0.05,1
4,2003,1113,1272,10,7,3,-0.17,1,-55,-2,...,1,1,0.04,0.02,-0.03,-3,4,0.05,0.02,1
5,2003,1141,1166,11,6,5,-0.09,0,-315,0,...,5,3,0.01,0.08,-0.01,-1,9,0.02,0.02,1
6,2003,1143,1301,8,9,-1,0.12,2,5,4,...,0,-1,0.01,-0.08,0.03,-1,1,-0.01,-0.01,1
7,2003,1163,1140,5,12,-7,-0.04,8,-13,7,...,2,-3,0.00,-0.08,0.01,4,8,-0.03,0.01,1
8,2003,1163,1390,5,4,1,-0.04,8,95,4,...,1,0,0.03,-0.01,0.03,5,5,-0.03,0.02,1
9,2003,1181,1161,3,14,-11,0.23,7,307,5,...,-2,1,-0.06,0.01,0.00,4,-2,-0.01,0.00,1


In [44]:
df_Concats.describe()

,Season,Wteam,Lteam,win_seed,loss_seed,seed_diff,diff_WinLossPct,diff_AvgPts,diff_PtDiff,diff_AvgRebounds,...,diff_AvgTurnovers,diff_AvgFouls,diff_FGPct,diff_FTPct,diff_ThPct,diff_AvgStealsPlusBlocks,diff_AvgPtsAgainst,diff_OppFGPct,diff_OppThPct,Result
count,1828.000000,1828.000000,1828.000000,1828.000000,1828.000000,1828.000000,1.828000e+03,1828.00000,1828.000000,1828.00000,...,1828.000000,1828.000000,1.828000e+03,1.828000e+03,1.828000e+03,1828.000000,1828.000000,1.828000e+03,1.828000e+03,1828.000000
mean,2009.578775,1294.508753,1294.598468,5.112691,8.805252,-3.692560,2.556914e-17,0.00000,0.000000,0.00000,...,0.000000,0.000000,6.194899e-18,-1.396889e-18,-1.852396e-18,0.000000,0.000000,2.125701e-19,1.245053e-18,0.500000
std,4.037974,102.358912,106.976007,3.982913,4.630086,6.507678,1.421625e-01,7.25054,205.540046,3.73375,...,2.147541,2.565187,3.211399e-02,5.158463e-02,3.787312e-02,2.981975,6.378501,3.126594e-02,3.153438e-02,0.500137
min,2003.000000,1104.000000,1102.000000,1.000000,1.000000,-15.000000,-6.300000e-01,-23.00000,-740.000000,-18.00000,...,-8.000000,-7.000000,-1.100000e-01,-1.700000e-01,-1.400000e-01,-9.000000,-25.000000,-1.200000e-01,-1.000000e-01,0.000000
25%,2006.000000,1211.000000,1208.000000,2.000000,5.000000,-8.000000,-9.000000e-02,-5.00000,-127.000000,-2.00000,...,-1.000000,-2.000000,-2.000000e-02,-4.000000e-02,-2.000000e-02,-2.000000,-4.000000,-2.000000e-02,-2.000000e-02,0.000000
50%,2010.000000,1277.000000,1295.000000,4.000000,9.000000,-3.500000,0.000000e+00,0.00000,0.000000,0.00000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.500000
75%,2013.000000,1393.000000,1393.000000,8.000000,13.000000,1.000000,9.000000e-02,5.00000,127.000000,2.00000,...,1.000000,2.000000,2.000000e-02,4.000000e-02,2.000000e-02,2.000000,4.000000,2.000000e-02,2.000000e-02,1.000000
max,2016.000000,1463.000000,1463.000000,16.000000,16.000000,13.000000,6.300000e-01,23.00000,740.000000,18.00000,...,8.000000,7.000000,1.100000e-01,1.700000e-01,1.400000e-01,9.000000,25.000000,1.200000e-01,1.000000e-01,1.000000


In [45]:
df_Concats['Result'].sum()

914